# Data Scraping
**CRIMES VIOLENTOS LETAIS E INTENCIONAIS – CVLI (SSPDS/CE)**  
>Autor:       Erivando Sena  
E-mail:      erivandosena@gmail.com   
Criado:        03/08/2019  
Atualizado:  14/02/2023  

### Setup packages

In [4]:
pacotes_padrao <- c("remotes","devtools","rJava")
pacotes_analise <- c("tidyverse","tabulizer","rvest","stringi","readxl","lubridate",
                     "leaflet","htmlwidgets","ggthemes","forecast","prophet", "mice",
                     "kableExtra", "ggmap","stringr","purrr","dplyr","stringr","pdftools", "readr")

suppressPackageStartupMessages({
  for (pkg in c(pacotes_padrao, pacotes_analise)) {
    if (!(pkg %in% rownames(installed.packages()))){
      install.packages(pkg)
    }
  }
  if (!require("remotes")) {
    remotes::install_github(c("ropensci/tabulizerjars", "ropensci/tabulizer"), INSTALL_opts = "--no-multiarch")
  }
  if (!require("rgdal")) {
    install.packages("rgdal", repos = "http://R-Forge.R-project.org")
  }
  if (!require("tabplot")) {
    remotes::install_github("mtennekes/tabplot")
  }
  for (pkg in c(pacotes_padrao, pacotes_analise)) {
    require(pkg, character.only = TRUE)
  }
})

Warning message:
"package 'remotes' was built under R version 4.3.2"
Warning message:
"package 'sp' was built under R version 4.3.2"
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'tabplot'"



Skipping 1 packages not available: ffbase



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file 'C:\Users\Erivando\AppData\Local\Temp\RtmpKa5X7z\remotes2102adb4296\mtennekes-tabplot-b3b876f/DESCRIPTION' ... OK
* preparing 'tabplot':
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* looking to see if a 'data/datalist' file should be added
* building 'tabplot_1.4.1.tar.gz'



Installing package into 'C:/Users/Erivando/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)

Warning message in i.p(...):
"installation of package 'C:/Users/Erivando/AppData/Local/Temp/RtmpKa5X7z/file2106f1e2f8c/tabplot_1.4.1.tar.gz' had non-zero exit status"
Warning message:
"package 'usethis' was built under R version 4.3.2"
Warning message:
"package 'rJava' was built under R version 4.3.2"
Warning message:
"package 'tidyverse' was built under R version 4.3.2"
Warning message:
"package 'ggplot2' was built under R version 4.3.2"
Warning message:
"package 'tibble' was built under R version 4.3.2"
Warning message:
"package 'tidyr' was built under R version 4.3.2"
Warning message:
"package 'readr' was built under R version 4.3.2"
Warning message:
"package 'purrr' was built under R version 4.3.2"
Warning message:
"package 'dplyr' was built under R version 4.3.2"
Warning message:
"package 'stringr' was built under R version 4.3.2"
Warning message:
"package 'forcats' was built und

### Folders configuration

In [5]:
dir_docs <- c("Documentos")
dir_dados <- c("Dados")
# Vetor contendo o camainho absoluto dos diretorios
diretorios <- c(dir_dados, dir_docs)
for (diretorio in diretorios) {
  if(!dir.exists(diretorio)) {
    dir.create(diretorio, recursive = TRUE)
  }
}

### Functions

In [6]:
# Extrai informacoes de ano do código html
extrai_lista_anos <- function(url_site) {
  pagina_html <- xml2::read_html(url_site)

  df_lista_urls <- pagina_html %>%
    rvest::html_nodes('.grid a.box') %>%
    purrr::map(xml2::xml_attrs) %>%
    purrr::map_df(~as.list(.)) %>%
    na.omit(.) %>%
    .[-c(nrow(.)),]

  df_lista_titulos <- pagina_html %>%
    rvest::html_nodes('.grid p') %>%
    rvest::html_text() %>%

    gsub(" ", "", .) %>%
    substr(., nchar(.)-3, nchar(.)) %>%
    .[-length(.)] %>% # remove ano 2013
    as.data.frame(.) %>%
    setNames(., "ano")

  # Juntar os dataframes df_lista_urls e df_lista_titulos
  df_urls_anos <- data.frame(df_lista_urls, df_lista_titulos)
  excluir <- c("class", "target")
  df_urls_anos <- df_urls_anos[,!(names(df_urls_anos) %in% excluir)]
  df_urls_anos[,1][df_urls_anos[, 1] == "#"] <- NA

  return(df_urls_anos)
}

# Baixa lista de arquivos por ano
obtem_arquivos <- function(url_site, anos) {
  pastas <- list.dirs(dir_docs, full.names = TRUE, recursive = FALSE)
  unlink(pastas, recursive = TRUE)

  for(indice in 1:nrow(anos)) {
    print(paste("Lendo codigo HTML da pagina Web:", anos[indice,1], "-", anos[indice,2], sep = " "))
    df_listas <- extrai_lista_documentos(anos[indice,1])
    Sys.sleep(1)
    download_documentos(df_listas)
    Sys.sleep(10)
  }
}

# Extrai informacoes de docs do codigo html
extrai_lista_documentos <- function(url_pagina) {
  pagina_html <- xml2::read_html(url_pagina)

  df_lista_urls <- pagina_html %>%
    rvest::html_nodes('.-Verde a.box') %>%
    purrr::map(xml2::xml_attrs) %>%
    purrr::map_df(~as.list(.))

  df_lista_titulos <- pagina_html %>%
    rvest::html_nodes('.-Verde h3') %>%
    rvest::html_text() %>%
    remove_acentos(.) %>%
    gsub("–", " - ", .) %>%
    gsub("/", " - ", .) %>%
    gsub(" ", "", .) %>%
    as.data.frame(.) %>%
    setNames(., "mes")

  # Juntar os dataframes df_lista_urls e df_lista_titulos
  df_urls_nomes <- data.frame(df_lista_urls, df_lista_titulos)
  excluir <- c("class", "target", "rel")
  df_urls_nomes <- df_urls_nomes[,!(names(df_urls_nomes) %in% excluir)]
  df_urls_nomes[,1][df_urls_nomes[, 1] == "#"] <- NA

  return(df_urls_nomes)
}

# Baixa os arquivos da web para uma pasta local
download_documentos <- function(d_frame_lista) {
  anos <- extrai_lista_anos(URL_site)
  for (titulo in d_frame_lista$mes) {
    df_url <- select(filter(d_frame_lista, d_frame_lista$mes == titulo), "href")
    df_nome <- select(filter(d_frame_lista, d_frame_lista$mes == titulo), "mes")
    for(ano in anos$ano) {
      if(ano == substr(df_nome$mes, nchar(as.character(df_nome$mes))-3, nchar(as.character(df_nome$mes)))) {
        diretorio <- file.path(".", dir_docs, ano)
        if(!dir.exists(diretorio)) {
          dir.create(diretorio, recursive = TRUE)
        }
        nome_arquivo_completo <- obtem_nome_arquivos(ano, df_nome$mes)
        if(file.exists(nome_arquivo_completo)) {
          print(paste0("Arquivo baixado: ", df_nome$mes))
        } else {
          if (!is.na(df_url$href)) {
            download.file(df_url$href, destfile = nome_arquivo_completo, mode = "wb", quiet=FALSE)
          } else {
            print(paste("Arquivo indisponivel: ", df_nome$mes, sep = " "))
          }
        }
      }
    }
  }
}

# Formata nome dos arquivos
obtem_nome_arquivos <- function(dir_superior, nome_relacao) {
  nome_arquivo <- nome_relacao %>% gsub("[/. ,]","_", .)
  path <- file.path(".", dir_docs, dir_superior, paste0(nome_arquivo, ".pdf"))
  return(path)
}

# Remover acentos
remove_acentos <- function(obj_str) {
  if(!is.character(obj_str)) {
    obj_str <- as.character(obj_str)
  }
  obj_str <- stringi::stri_trans_general(str = obj_str, "latin-ascii")
  return(obj_str)
}

# Aplicar transformações em data frame
processar_dataframe <- function(df) {
  df <- df %>% as.data.frame()
  df <- df[,-1]
  if (ncol(df) >= 7) {
    df <- df %>%
      select(1:7) %>%
      setNames(cabecalho)
      df <- df %>% mutate_all(~ifelse(. == "", NA, .))
      df <- na.omit(df)
      df <- df %>% mutate(across(everything(), as.character))
    return(df)
  }
}

### Collect of data

In [7]:
URL_site <- "https://www.sspds.ce.gov.br/estatisticas-2-2-2-2-2-2/"
anos <- extrai_lista_anos(URL_site)

anos_filtro <- anos[anos$ano %in% c("2023","2022","2021","2020"), ] # Year 2023 only, keep c("2023")
# anos_filtro <- anos # Enabled all years

obtem_arquivos(URL_site, anos_filtro)
vetor_anos <- as.numeric(anos_filtro$ano)
lista_df <- list()

for (iano in seq_along(vetor_anos)) {
  data_frame_meses <- extrai_lista_documentos(anos_filtro[iano, 1])
  num_docs <- nrow(data_frame_meses)
  for (ndoc in 1:num_docs) {
    arquivo <- file.path(dir_docs, vetor_anos[iano], paste0(data_frame_meses[ndoc, 2], ".pdf"))
    paginas <- pdf_info(arquivo)$pages
    for (npage in 1:paginas) {
      tabela <- extract_tables(file = arquivo, pages = npage, method = "stream", guess = TRUE, output = "data.frame", encoding = "UTF-8")
      lista_df[[length(lista_df) + 1]] <- tabela
    }
  }
}

[1] "Lendo codigo HTML da pagina Web: https://www.sspds.ce.gov.br/estatisticas-2023 - 2023"
[1] "Lendo codigo HTML da pagina Web: https://www.sspds.ce.gov.br/estatisticas-2022 - 2022"
[1] "Lendo codigo HTML da pagina Web: https://www.sspds.ce.gov.br/estatisticas-2021 - 2021"
[1] "Lendo codigo HTML da pagina Web: https://www.sspds.ce.gov.br/estatisticas-2020 - 2020"


### Processing of data

In [202]:
# Clear data and merge data frame
arquivos_csv <- list.files(dir_dados, pattern = "\\.csv$", full.names = TRUE)
file.remove(arquivos_csv)

df_final <- data.frame()
cabecalho = c("AIS","MUNICIPIO_HOMICIDIO", "NATUREZA_HOMICIDIO", "ARMA_UTILIZADA", "DATA_HOMICIDIO", "SEXO", "IDADE")
lista_df_processada <- lapply(lista_df, processar_dataframe)

for (i in seq_along(lista_df_processada)) {
  df <- lista_df_processada[[i]]
  df_final <- bind_rows(df_final, df)
  # write.csv(df, file = paste0(file.path(dir_dados, paste0(i,"cvli.csv"))), row.names = FALSE) # Enable export the csv of data frames unitary
}

df_final <- df_final %>% arrange(as.Date(DATA_HOMICIDIO, format="%d/%m/%Y")) %>% mutate(ID = c(1:nrow(.))) #%>% .[,c(8,1:ncol(.))]
df_final <- df_final %>% .[,c(8,1:ncol(.)-1)]

# Export dataset
write.csv(df_final, file = paste0(file.path(dir_dados, "cvli.csv")), row.names = FALSE)

[1] TRUE

ID,AIS,MUNICIPIO_HOMICIDIO,NATUREZA_HOMICIDIO,ARMA_UTILIZADA,DATA_HOMICIDIO,SEXO,IDADE
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,AIS 12,Maranguape,HOMICIDIO DOLOSO,Arma de fogo,01/01/2020,Masculino,26
2,AIS 11,Caucaia,HOMICIDIO DOLOSO,Outros meios,01/01/2020,Masculino,40
3,AIS 19,Mauriti,HOMICIDIO DOLOSO,Arma branca,01/01/2020,Masculino,23
4,AIS 11,Caucaia,HOMICIDIO DOLOSO,Arma branca,01/01/2020,Masculino,53
5,AIS 6,Fortaleza,HOMICIDIO DOLOSO,Outros meios,01/01/2020,Masculino,19
6,AIS 2,Fortaleza,HOMICIDIO DOLOSO,Arma branca,01/01/2020,Feminino,18
7,AIS 16,Santa Quitéria,HOMICIDIO DOLOSO,Arma de fogo,01/01/2020,Masculino,17
8,AIS 14,Forquilha,HOMICIDIO DOLOSO,Arma de fogo,01/01/2020,Masculino,32
9,AIS 6,Fortaleza,HOMICIDIO DOLOSO,Arma de fogo,01/01/2020,Feminino,27


### Analysis of dataset

In [204]:
# Import dataset
dados_cvli <- read_csv(file = paste0(file.path(dir_dados, "cvli.csv")), show_col_types = FALSE)
print(dados_cvli)

# A tibble: 12,599 × 8
      ID AIS    MUNICIPIO_HOMICIDIO NATUREZA_HOMICIDIO ARMA_UTILIZADA
   <dbl> <chr>  <chr>               <chr>              <chr>         
 1     1 AIS 12 Maranguape          HOMICIDIO DOLOSO   Arma de fogo  
 2     2 AIS 11 Caucaia             HOMICIDIO DOLOSO   Outros meios  
 3     3 AIS 19 Mauriti             HOMICIDIO DOLOSO   Arma branca   
 4     4 AIS 11 Caucaia             HOMICIDIO DOLOSO   Arma branca   
 5     5 AIS 6  Fortaleza           HOMICIDIO DOLOSO   Outros meios  
 6     6 AIS 2  Fortaleza           HOMICIDIO DOLOSO   Arma branca   
 7     7 AIS 16 Santa Quitéria      HOMICIDIO DOLOSO   Arma de fogo  
 8     8 AIS 14 Forquilha           HOMICIDIO DOLOSO   Arma de fogo  
 9     9 AIS 6  Fortaleza           HOMICIDIO DOLOSO   Arma de fogo  
10    10 AIS 15 Aracoiaba           HOMICIDIO DOLOSO   Outros meios  
# ℹ 12,589 more rows
# ℹ 3 more variables: DATA_HOMICIDIO <chr>, SEXO <chr>, IDADE <chr>
